**Loading of data**

Initial code taken from: https://github.com/pierinim/tutorials/blob/master/GGI_Jan2021/Lecture1/Notebook1_ExploreDataset.ipynb

In [2]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

In [3]:
A = ! ls tutorials/Data/JetDataset
fileIN =  f'tutorials/Data/JetDataset/{A[2]}'
print(fileIN)
print(A)

tutorials/Data/JetDataset/jetImage_7_100p_30000_40000.h5
['jetImage_7_100p_0_10000.h5', 'jetImage_7_100p_10000_20000.h5', 'jetImage_7_100p_30000_40000.h5', 'jetImage_7_100p_40000_50000.h5', 'jetImage_7_100p_50000_60000.h5', 'jetImage_7_100p_60000_70000.h5', 'jetImage_7_100p_70000_80000.h5', 'jetImage_7_100p_80000_90000.h5']


In [4]:
file = h5py.File(fileIN)

In [5]:
print(list(file.keys()))

['jetConstituentList', 'jetFeatureNames', 'jetImage', 'jetImageECAL', 'jetImageHCAL', 'jets', 'particleFeatureNames']


In [6]:
jetconstituents = file.get('jetConstituentList')


# Shape: num_jets x num_particles x 4
fourvectors = jetconstituents[:, :, :4] # Particles (px, py, pz, E)


In [7]:
# Creating full matrix of inner products
M = np.diag([-1, -1, -1, 1])
inner_prods = np.einsum("npi, ij, nqj->npq", fourvectors, M, fourvectors)
print(inner_prods.shape)

(10000, 100, 100)


In [8]:
jet_data = np.array(file.get('jets'))
target = jet_data[:, -6:-1]
data = np.expand_dims(inner_prods, axis=-1)
print(f'Target shape: {target.shape} Data shape: {data.shape}')


Target shape: (10000, 5) Data shape: (10000, 100, 100, 1)


**Using the PELICAN architecture**

In [9]:
import keras
import tensorflow as tf
from pelican import PELICAN

In [10]:

model = PELICAN(
    depth=2,
    activation='leaky_relu',
    msg_outputs=10,
    agg_outputs=20,
    scal_outputs=10,
    dense_output=5
)


# model.compile(
#     loss='categorical_crossentropy',
#     optimizer='adam',
#     metrics=['accuracy']
# )

In [11]:
y = model(data[:100])

In [12]:
print(y)

tf.Tensor(
[[0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]

In [13]:
# https://www.tensorflow.org/tutorials/customization/custom_training_walkthrough

loss_fn = keras.losses.CategoricalCrossentropy()
def loss(model, x, y, training):
    res = model(x, training=training)
    return loss_fn(y_true=y, y_pred=res)

def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets, training=True)
    return loss_value, tape.gradient(loss_value, model.trainable_variables)

optimizer = tf.keras.optimizers.Adam()

In [14]:
EPOCHS = 10
BATCH = 128
numsamples = 10000

In [15]:
def make_batches(data, target, batch_size):
    assert(data.shape[0] == target.shape[0])
    return [
        (data[N:(N+batch_size)], target[N:(N+batch_size)]) for N in range(0, data.shape[0], batch_size)
    ]

In [16]:

batches = make_batches(data[:numsamples], target[:numsamples], BATCH)


In [17]:
from tqdm import tqdm

In [18]:
train_loss_results = []
train_accuracy_results = []
for epoch in range(EPOCHS):
    epoch_loss_avg = keras.metrics.Mean()
    epoch_accuracy = keras.metrics.CategoricalAccuracy()

    with tqdm(total=len(batches)) as pbar:
        for x, y in batches:
            loss_value, grads = grad(model, x, y)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
            pbar.update(1)
        epoch_loss_avg.update_state(loss_value)
        epoch_accuracy.update_state(y, model(x, training=True))

    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())

    print(f'Epoch: {epoch} Loss: {epoch_loss_avg.result()} Accuracy: {epoch_accuracy.result()}')




  0%|          | 0/79 [00:00<?, ?it/s]

In [ ]:
y = model(data[:100])

In [ ]:
print(y)

tf.Tensor(
[[0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]